<a href="https://colab.research.google.com/github/clghks92/-/blob/master/KW_MMDS_Colab_1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KW_MMDS - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 57kB/s 
     |████████████████████████████████| 204kB 22.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=102bb3ee44d458cb0d2401097e8a48e605c8f466e2f00ee78872345c25c7f2b1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

# 새 섹션

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 새 섹션

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

1. Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)

#RDD 파일로 읽어오기
text = sc.textFile('pg100.txt')

words = text.map(lambda line: line.lower()) #소문자로 전환


wordResult = words.flatMap(lambda line: line.split()) #단어 나누기

wordResult2 = wordResult.filter(lambda line: line[0].isalpha()) #알파벳으로 시작하는 단어
wordResult3 = wordResult2.flatMap(lambda line: line[:1]) #첫 문자만 남기기

wordTuples = wordResult3.map(lambda line: (line, 1)) #tuples 만들기


wordCounts = wordTuples.reduceByKey(lambda a, b: a + b).map(lambda tuple: (tuple[1], tuple[0]))




wordLast = wordCounts.sortByKey(ascending=False)
wordLast.take(80)








[(123602, 't'),
 (84836, 'a'),
 (65705, 's'),
 (62167, 'i'),
 (60563, 'h'),
 (59597, 'w'),
 (55676, 'm'),
 (45455, 'b'),
 (43494, 'o'),
 (36814, 'f'),
 (34567, 'c'),
 (29713, 'd'),
 (29569, 'l'),
 (27759, 'p'),
 (26759, 'n'),
 (25855, 'y'),
 (20782, 'g'),
 (18697, 'e'),
 (14265, 'r'),
 (9418, 'k'),
 (9170, 'u'),
 (5728, 'v'),
 (3339, 'j'),
 (2377, 'q'),
 (71, 'z'),
 (14, 'x')]

2. Write a Spark application which outputs the 10 most frequently used words contained in 'the Pride and Prejudice' by Jane Austen. In your implementation **ignore the letter case**, i.e., consider all words as **upper** case.

In [ ]:
# 오만과 편견 다운로드: 1342-0.txt
pride_id='11HHiWgRiXHuNehW8AXrHarKJfGX1HHnB'
pride_downloaded = drive.CreateFile({'id': pride_id})
pride_downloaded.GetContentFile('1342-0.txt')

In [ ]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)

text = sc.textFile('1342-0.txt')

words = text.map(lambda line: line.lower()) #소문자로 전환
words.take(10)

wordResult = words.flatMap(lambda line: line.split()) #단어 나누기

wordResult2 = wordResult.filter(lambda line: line[0].isalpha()) #알파벳만 남기기


wordTuples = wordResult2.map(lambda line: (line, 1)) #tuples 만들기

wordCounts = wordTuples.reduceByKey(lambda a, b: a + b).map(lambda tuple: (tuple[1], tuple[0]))


wordLast = wordCounts.sortByKey(ascending=False)
wordLast.take(10)


[(4480, 'the'),
 (4169, 'to'),
 (3681, 'of'),
 (3400, 'and'),
 (1981, 'a'),
 (1939, 'her'),
 (1890, 'in'),
 (1797, 'was'),
 (1725, 'i'),
 (1607, 'she')]